In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, concatenate, Conv2DTranspose
from tensorflow.keras.models import Model
import os
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
input_shape1 = (256, 256, 3)
input_shape2 = (256, 256, 3)
output_shape = (256, 256, 3)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define input layers
input1 = Input(shape=input_shape1, name='input1')
input2 = Input(shape=input_shape2, name='input2')

conv1_1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input1)
conv1_2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1_1)
pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1_2)

conv2_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2_1)
pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2_2)

conv3_1 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3_2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3_1)

# Define convolutional layers for input 2
conv4_1 = Conv2D(32, (3, 3), activation='relu', padding='same')(input2)
conv4_2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv4_1)
pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4_2)

conv5_1 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool4)
conv5_2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv5_1)
pool5 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv5_2)

conv6_1 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool5)
conv6_2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv6_1)

# Concatenate convolutional layers from both inputs
concat = concatenate([conv3_2, conv6_2])

# Define deconvolutional layers for output
deconv1 = Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same')(concat)
deconv2 = Conv2DTranspose(32, (3, 3), strides=(2, 2), activation='relu', padding='same')(deconv1)
output = Conv2DTranspose(3, (3, 3), activation='softmax', padding='same')(deconv2)

# Define the model with two inputs and one output
model = Model(inputs=[input1, input2], outputs=output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input1 (InputLayer)            [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 input2 (InputLayer)            [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_12 (Conv2D)             (None, 256, 256, 32  896         ['input1[0][0]']                 
                                )                                                           

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
)

In [ ]:
def image_to_array(folder_path):
    img_array=[]
    file_list = os.listdir(folder_path)
    for file in file_list:
        img = plt.imread(os.path.join(folder_path, file))
        img_array.append(np.array(img))
    return img_array

In [ ]:
input1 = image_to_array('/content/drive/MyDrive/fishing dataset/dataset/temp-dataset-256')
input1 = np.array(input1)
input1=input1[:,:,:,:3]

input2 = image_to_array('/content/drive/MyDrive/fishing dataset/dataset/chl-dataset-256')
input2 = np.array(input2)
input2=input2[:,:,:,:3]

target = image_to_array('/content/drive/MyDrive/fishing dataset/dataset/target_dataset')
target = np.array(target)
target = target[:,:,:,:3]

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse', metrics=['mean_absolute_error'])

In [ ]:
#checkpoint_path = "/content/drive/MyDrive/checkpoints/cp.ckpt"
#checkpoints = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, save_best_only=True, monitor='val_loss', verbose=1)

In [ ]:
model.fit(train_datagen.flow([input1, input2], target, batch_size=20), epochs=100)

Epoch 1/100
40/40 [==============================] - 36s 415ms/step - loss: 0.0435 - mean_absolute_error: 0.0821
Epoch 2/100
40/40 [==============================] - 17s 420ms/step - loss: 0.0125 - mean_absolute_error: 0.0125
Epoch 3/100
40/40 [==============================] - 16s 391ms/step - loss: 0.0125 - mean_absolute_error: 0.0125
Epoch 4/100
40/40 [==============================] - 16s 394ms/step - loss: 0.0125 - mean_absolute_error: 0.0125
Epoch 5/100
40/40 [==============================] - 16s 399ms/step - loss: 0.0125 - mean_absolute_error: 0.0125
Epoch 6/100
40/40 [==============================] - 16s 401ms/step - loss: 0.0125 - mean_absolute_error: 0.0125
Epoch 7/100
40/40 [==============================] - 16s 404ms/step - loss: 0.0125 - mean_absolute_error: 0.0125
Epoch 8/100
40/40 [==============================] - 16s 391ms/step - loss: 0.0125 - mean_absolute_error: 0.0125
Epoch 9/100
40/40 [==============================] - 16s 393ms/step - loss: 0.0125 - mean_absolu

In [ ]:
model.save("/content/drive/MyDrive/fishing dataset/model.h5")